In [1]:
import chardet
import pandas as pd
import numpy as np
from scipy.spatial import distance
from pyproj import Transformer
from geopy.distance import geodesic
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import folium

In [2]:

승우_광장동_df = pd.read_csv("/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/입지분석2/유성/군집화결과/clustered_coordinates(Gwangjang-dong).csv")   # cluster : 3
유성_knn광장동_df = pd.read_csv("/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/입지분석2/유성/군집화결과/knn_cluster_광장동.csv")                    # cluster : 2 
유성_gnn광장동_df = pd.read_csv("/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/입지분석2/유성/군집화결과/gmm_cluster_광장동.csv")                   # clutser : 2 or 5

승우_자양2동_df = pd.read_csv("/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/입지분석2/유성/군집화결과/clustered_coordinates(Jayang2_dong).csv")    # clutser : 2
유성_knn자양2동_df = pd.read_csv("/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/입지분석2/유성/군집화결과/knn_cluster_자양2동.csv")                  # clutser : 3
유성_gnn자양2동_df = pd.read_csv("/Users/yujin/Desktop/파일/3-1/데이터분석 공모전/codes/입지분석2/유성/군집화결과/gmm_cluster_자양2동.csv")                 # clutser : 1

# 목표1 : (승우_광장동_df[cluster]== 3 )and (유성_knn광장동_df[cluster] == 2 or 5 ) and (유성_gnn광장동_df[cluster]==2 ) 교집합 데이터 찾아서 새로운 df반환
# 목표1 : (승우_자양2동_df[cluster]== 2 )and (유성_knn자양2동_df[cluster] == 3 ) and (유성_gnn자양2동_df[cluster]==1 ) 교집합 데이터 찾아서 새로운 df반환


In [3]:
승우_광장동_df.head(1)

,행정동,경도,위도,subway_최소거리(m),500m지하철역개수,bus_최소거리(m),500m버스정류장개수,badplace_최소거리(m),500m술집개수,parking_최소거리(m),500내주차장개수,300m보호구역여부,200m위험구역여부,cluster
0,광장동,127.094406,37.543013,532.519994,0,78.42608,5,160.337223,22,425.357698,0,1,0,2


In [4]:
승우_자양2동_df.head(1)

,행정동,경도,위도,subway_최소거리(m),500m지하철역개수,bus_최소거리(m),500m버스정류장개수,badplace_최소거리(m),500m술집개수,parking_최소거리(m),500내주차장개수,300m보호구역여부,200m위험구역여부,cluster
0,자양2동,127.073006,37.524009,952.789593,0,491.107127,0,512.930392,0,271.192001,0,0,0,1


In [5]:
유성_knn광장동_df.head(1)

,Unnamed: 0.1,Unnamed: 0,행정동,경도,위도,subway_최소거리(m),500m지하철역개수,bus_최소거리(m),500m버스정류장개수,badplace_최소거리(m),500m술집개수,parking_최소거리(m),500내주차장개수,300m보호구역여부,200m위험구역여부,cluster
0,1218,1218,광장동,127.094406,37.543013,532.519994,0,78.42608,5,160.337223,22,425.357698,0,1,0,0


In [6]:
유성_knn자양2동_df.head(1)

,Unnamed: 0.1,Unnamed: 0,행정동,경도,위도,subway_최소거리(m),500m지하철역개수,bus_최소거리(m),500m버스정류장개수,badplace_최소거리(m),500m술집개수,parking_최소거리(m),500내주차장개수,300m보호구역여부,200m위험구역여부,cluster
0,263,263,자양2동,127.073006,37.524009,952.789593,0,491.107127,0,512.930392,0,271.192001,0,0,0,1


In [7]:
유성_gnn자양2동_df.head(1)

,Unnamed: 0.1,Unnamed: 0,행정동,경도,위도,subway_최소거리(m),500m지하철역개수,bus_최소거리(m),500m버스정류장개수,badplace_최소거리(m),500m술집개수,parking_최소거리(m),500내주차장개수,300m보호구역여부,200m위험구역여부,cluster
0,263,263,자양2동,127.073006,37.524009,952.789593,0,491.107127,0,512.930392,0,271.192001,0,0,0,2


In [8]:
유성_gnn광장동_df.head(1)

,Unnamed: 0.1,Unnamed: 0,행정동,경도,위도,subway_최소거리(m),500m지하철역개수,bus_최소거리(m),500m버스정류장개수,badplace_최소거리(m),500m술집개수,parking_최소거리(m),500내주차장개수,300m보호구역여부,200m위험구역여부,cluster
0,1218,1218,광장동,127.094406,37.543013,532.519994,0,78.42608,5,160.337223,22,425.357698,0,1,0,2


In [ ]:
# ===== 목표1 : 광장동 교집합 =====

# 필요한 조건에 맞게 필터링
condition_광장동 = (
    (승우_광장동_df['cluster'] == 3) &
    (유성_knn광장동_df['cluster'].isin([2, 5])) &
    (유성_gnn광장동_df['cluster'] == 2)
)

# 조건을 모두 만족하는 인덱스를 기준으로 새로운 데이터프레임 생성
df_광장동 = 승우_광장동_df[condition_광장동].copy()

# ===== 목표2 : 자양2동 교집합 =====

condition_자양2동 = (
    (승우_자양2동_df['cluster'] == 4) & # 3->4 로 변환
    (유성_knn자양2동_df['cluster'] == 3) &
    (유성_gnn자양2동_df['cluster'] == 1)
)

df_자양2동 = 승우_자양2동_df[condition_자양2동].copy()

In [10]:
df_광장동

,행정동,경도,위도,subway_최소거리(m),500m지하철역개수,bus_최소거리(m),500m버스정류장개수,badplace_최소거리(m),500m술집개수,parking_최소거리(m),500내주차장개수,300m보호구역여부,200m위험구역여부,cluster
85,광장동,127.102301,37.548448,254.106988,1,107.252166,5,165.342168,7,155.554862,2,1,0,3
106,광장동,127.103443,37.546649,91.522946,1,84.794284,6,27.998259,9,202.695208,2,1,0,3
107,광장동,127.103438,37.547551,151.661287,1,119.594094,7,75.138358,10,215.423397,2,1,0,3
108,광장동,127.103433,37.548452,212.023355,1,152.938334,6,133.550156,9,223.039760,2,1,0,3
129,광장동,127.104575,37.546653,143.967268,1,91.957582,8,46.159112,11,163.095180,2,1,0,3
199,광장동,127.107952,37.550270,590.923619,0,44.236036,7,184.717723,6,301.383160,1,0,0,3


In [11]:
df_자양2동

,행정동,경도,위도,subway_최소거리(m),500m지하철역개수,bus_최소거리(m),500m버스정류장개수,badplace_최소거리(m),500m술집개수,parking_최소거리(m),500내주차장개수,300m보호구역여부,200m위험구역여부,cluster
95,자양2동,127.085425,37.529462,514.101577,0,206.757908,3,152.871864,12,239.109558,2,1,0,4
96,자양2동,127.085420,37.530363,454.053266,0,146.329819,4,111.642799,12,184.922852,2,1,0,4
97,자양2동,127.085415,37.531264,394.145183,0,86.011650,5,66.820304,17,131.393129,3,1,0,4
98,자양2동,127.085410,37.532166,334.452685,0,26.562950,5,25.600193,21,87.486995,3,1,0,4
108,자양2동,127.086557,37.529466,515.881963,0,243.640709,3,238.101519,11,122.715938,1,1,0,4
109,자양2동,127.086552,37.530367,455.948590,0,185.172086,4,195.271796,12,104.660377,2,1,0,4
110,자양2동,127.086547,37.531268,396.189776,0,128.604409,5,163.987733,12,119.041785,2,1,0,4
111,자양2동,127.086542,37.532170,336.698470,0,78.179072,5,151.575178,17,73.351261,2,1,0,4
121,자양2동,127.087688,37.529470,547.462131,0,248.772416,3,333.288130,9,68.492288,1,1,0,4
122,자양2동,127.087684,37.530371,491.285416,0,191.592726,4,303.994522,10,21.870792,1,1,0,4


In [12]:
import folium

# ---------------------------
# 광장동 지도 만들기
# ---------------------------

# 광장동 지도 초기화 (첫 포인트 기준으로 중심 잡음)
m_gwangjang = folium.Map(
    location=[df_광장동['위도'].mean(), df_광장동['경도'].mean()],
    zoom_start=15
)

# 광장동 데이터 마커 추가
for idx, row in df_광장동.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"광장동 {idx}",
        icon=folium.Icon(color='blue')
    ).add_to(m_gwangjang)

# 광장동 지도 저장
m_gwangjang.save('광장동_지도.html')

# ---------------------------
# 자양2동 지도 만들기
# ---------------------------

m_jayang2 = folium.Map(
    location=[df_자양2동['위도'].mean(), df_자양2동['경도'].mean()],
    zoom_start=15
)

for idx, row in df_자양2동.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"자양2동 {idx}",
        icon=folium.Icon(color='red')
    ).add_to(m_jayang2)

# 자양2동 지도 저장
m_jayang2.save('자양2동_지도.html')


In [13]:
df_광장동.to_csv("광장동_교집합.csv")
df_자양2동.to_csv("자양2동_교집합.csv")